In [2]:
import pandas as pd
import time
from openai import OpenAI
from dotenv import load_dotenv
from google import genai
import os
import json
from pydantic import BaseModel

from FetchDataFromDatabase.MainFetchDataFunction import fetch_data_from_database
from GetContentFromWebsite.MainGetContentFunction import fetch_content_from_website
from CleanContent.MainCleanContentFunction import clean_content
from GlobalFunctions.CreateClient import create_supabase_client

supabase = create_supabase_client()

### Daten aus Supabase fetchen

In [3]:
# DataFrame alle Lokalitäten aus Datenbank abfragen
fetched_localities_df = fetch_data_from_database()

2025-05-26 21:11:59,351:INFO - HTTP Request: GET https://pzxkdkuejjfgxgtkbubm.supabase.co/rest/v1/Locality?select=locality_id%2Cname%2Cemail_from_osm%2Cwebsite&error_code=is.null&has_discounts=is.null&email_sent=is.false&website=not.is.null "HTTP/1.1 200 OK"


In [ ]:
class ResponseSchema(BaseModel):
    has_discounts: bool
    summarized_discounts: str

def call_api(structured, prompt, input_text, provider,json_format=None, max_attempts=3):

    load_dotenv()
    attempts = 0
    while attempts < max_attempts:
        if provider == "deepseek":
            try:
                DEEPSEEK_API_KEY = os.environ.get("DEEPSEEK_API_KEY")
                client = OpenAI(api_key=DEEPSEEK_API_KEY, base_url="https://api.deepseek.com")
                response = client.chat.completions.create(
                    model="deepseek-chat",
                    messages=[
                        {"role": "system", "content": prompt},
                        {"role": "user", "content": input_text}
                    ],
                    stream=False
                )
                return response.choices[0].message.content  
            except Exception as e:
                attempts += 1
                print(f"API-Error Deepseek (Attempt {attempts}/{max_attempts}): {e}")
                time.sleep(2)  
        elif provider == "gemini":
            try: 
                GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")
                client = genai.Client(api_key=GOOGLE_API_KEY)
                if structured:
                    response = client.models.generate_content(
                        model="gemini-2.0-flash", 
                        contents=prompt + input_text,
                        config={
                            "response_mime_type": "application/json",
                            "response_schema": ResponseSchema,
                        })
                    return response.parsed
                else:
                    response = client.models.generate_content(
                        model="gemini-2.0-flash", 
                        contents=prompt + input_text)
                    return response.text

            except Exception as e:
                attempts += 1
                print(f"API-Error Gemini (Attempt {attempts}/{max_attempts}): {e}")
                time.sleep(2)
        elif provider == "openai":
            try:
                OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
                client = OpenAI(api_key=OPENAI_API_KEY)

                completion = client.chat.completions.create(
                model="gpt-4o-2024-08-06",
                store=True,
                messages=[
                    {"role": "system", "content": prompt},
                    {"role": "user", "content": input_text}
                ],
                response_format=json_format
                )

                data_as_json = completion.choices[0].message.content
                try:
                    return json.loads(data_as_json)
                except json.JSONDecodeError as e:
                    print(f"Error decoding JSON: {e}")
                    return None
            except Exception as e:
                attempts += 1
                print(f"API-Error OpenAI (Attempt {attempts}/{max_attempts}): {e}")
                time.sleep(2)

    print("Max attempts reached. Skipping this request.")
    return None

In [ ]:
discount_extraction_from_website_prompt_GEMINI= '''Extrahiere alle Informationen zu Rabatten für Menschen mit Behinderungen, einschließlich möglicher Vergünstigungen für Begleitpersonen und dem normalen Eintrittspreis (in der Regel für Erwachsene).
Achte darauf, dass es um individuelle Rabatte für Menschen mit Behinderung geht und keine allgemeinen Ermäßigungen für Gruppen, Senioren oder Studenten relevant sind. Das Wort „ermäßigt“ muss nicht für Beeinträchtigte stehen.
Gebe immer alle verschiedenen Eintrittsoptionen an. Die können lauten „Tageskarte“, „Jahreskarte“, „2h Sauna“, „Abend-Ticket“, usw…
Falls spezielle Merkzeichen (B, H, G, aG, Bl, Gl, TB) oder ein Mindestgrad der Behinderung (GdB) als Voraussetzung genannt sind, erfasse diese Informationen. Wenn kein GdB genannt wird, schreibe explizit, dass keiner angegeben ist.
Falls Preise für Begleitpersonen genannt werden, beachte: 
- Gibt es freie Eintritte für Begleiter? 
- Gibt es prozentuale Rabatte für Begleiter? 
Liefere alle gefundenen Rohdaten und Preisstrukturen in einer strukturierten und übersichtlichen Form. 
Gib auch keine Informationen zu Gruppen-Rabatten an, auch wenn es sich um eingeschränkte Gruppen handelt. Es geht nur um Einzelpersonen.
Wenn es für alle Besucher gratis ist oder nur nach einer optionalen Spende gefragt wird, gibt es dort keine Rabatte. 
Versehrte (Menschen mit Kriegsverletzungen) sind hier nicht als klassiche Beinträchtigte zu klassifizieren.
Falls gar keine Rabatte für Menschen mit Behinderung erwähnt sind, schreibe nur "AAA: Nein, es GIBT KEINE Rabatte für Menschen mit Einschränkungen."
Hier dein Eingabetext:
'''

validation_json_format = {
    "type": "json_schema",
    "json_schema": {
        "name": "discount_validation",
        "schema": {
            "type": "object",
            "properties": {
                "has_discounts_for_disabled": {
                    "type": "boolean",
                    "description": "Prüfe, ob es Rabatte für Menschen mit Behinderung gibt. Achte darauf, nur Rabatte für Menschen mit Behinderung zu berücksichtigen und keine allgemeinen Rabatte für Gruppen, Senioren oder Studenten."
                }
            },
            "required": ["has_discounts_for_disabled"],
            "additionalProperties": False
        },
        "strict": True
    }
}

extraction_json_format = {
    "type": "json_schema",
    "json_schema": {
        "name": "discount_extraction",
        "schema": {
            "type": "object",
            "properties": {
                "discount_options": {
                    "type": "array",
                    "description": "Liste aller verfügbaren Rabattoptionen für Menschen mit Behinderung.",
                    "items": {
                        "type": "object",
                        "properties": {
                            "name_of_option": {
                                "type": "string",
                                "description": "Name der Rabattoption (z. B. 'Tageskarte Sauna', '2h Spaßbad' oder 'Kinoabend')."
                            },
                            "degree_of_disability": {
                                "type": "integer",
                                "nullable": True,
                                "description": "Erforderlicher Grad der Behinderung (GdB). Falls keine explizite Angabe, wird 50 angenommen. Der Begriff 'Schwerbindert' bedeutet 50."
                            },
                            "mark_ag": {
                                "type": "integer",
                                "description": "Merkzeichen 'aG' (außergewöhnlich gehbehindert). Falls erforderlich, 1, sonst 0."
                            },
                            "mark_b": {
                                "type": "integer",
                                "description": "Merkzeichen 'B' (Begleitperson erforderlich). Falls erforderlich, 1, sonst 0."
                            },
                            "mark_bl": {
                                "type": "integer",
                                "description": "Merkzeichen 'Bl' (blind). Falls erforderlich, 1, sonst 0."
                            },
                            "mark_g": {
                                "type": "integer",
                                "description": "Merkzeichen 'G' (gehbehindert). Falls erforderlich, 1, sonst 0."
                            },
                            "mark_gl": {
                                "type": "integer",
                                "description": "Merkzeichen 'Gl' (gehörlos). Falls erforderlich, 1, sonst 0."
                            },
                            "mark_h": {
                                "type": "integer",
                                "description": "Merkzeichen 'H' (hilflos). Falls erforderlich, 1, sonst 0."
                            },
                            "mark_tb": {
                                "type": "integer",
                                "description": "Merkzeichen 'TBl' (taubblind). Falls erforderlich, 1, sonst 0."
                            },
                            "standard_price": {
                                "type": "number",
                                "description": "Regulärer Preis für das Angebot ohne Ermäßigung. Hier handelt es sich in der Regel um den Preis für Erwachsene."
                            },
                            "discounted_price": {
                                "type": "number",
                                "description": "Reduzierter Preis für Menschen mit Behinderung."
                            },
                            "companion_price": {
                                "type": "number",
                                "nullable": True,
                                "description": "Preis für eine Begleitperson, falls ein Rabatt gewährt wird. Falls kein Begleitpersonenrabatt existiert, ist der Wert 'null'."
                            }
                        },
                        "required": ["name_of_option", "degree_of_disability", "mark_ag", "mark_b", "mark_bl", "mark_g", "mark_gl", "mark_h", "mark_tb", "standard_price", "discounted_price", "companion_price"],
                        "additionalProperties": False
                    }
                }
            },
            "required": ["discount_options"],
            "additionalProperties": False
        },
        "strict": True
    }
}




validate_gemini_response_prompt_CHATGPT = '''Bitte validiere aus dem folgenden Text, ob dort Rabatte für Eingeschränkte vorkommen. Bitte sei dir sicher und antworte nur in dem gewollten JSON-Format. Falls es welche gibt, gib auch die Anzahl aus, wie viele Eintrittsoptionen es für Eingeschränkte gibt. 
Ein Beispiel: Sportbad 2h, Sportbad 4h, Spaßbad 2h. Hier wären es drei Eintrittsoptionen. Sei dir sicher, dass sich diese nur auf Eingeschränkte beziehen.'''

discount_extraction_from_api_response_prompt_CHATGPT = '''Basierend auf dem folgenden Text erstelle eine Ausgabe mit allen Rabattoptionen bzw. Eintrittsoptionen für Eingeschränkte im geforderten JSON-Format. 
Wichtige Regeln: 
1. Jede Rabattoption wird als eigenes JSON-Objekt im Array erwartet. 
2. Begleitpersonen-Rabatte dürfen nur in companion_price Feld erscheinen, niemals als eigenes JSON-Objekt. 
3. Die Namen, die du den Rabattoptionen gibst, sollen sich nicht auf den Namen der Lokalität beziehen oder diesen nennen. Diese Info wird separat angefügt. Nutze beschreibende Begriffe wie "Eintritt", wenn es nur eine Art des Eintritts gibt. Wenn es bsp. Jahreskarten, Tageskarte oder separate Eintrittsoptionen für separate Teile der Lokalität gibt, bennene dies sinnvoll.
Beispielausgaben: 
Eingabetext: 
"Tageskarte: Erwachsene 10€, Ermäßigt für Eingeschränkte (ab 50 GdB) 7€, Begleitperson frei" 
Korrekte Ausgabe (hier als CSV, soll aber als JSON ausgegeben werden): 
Tageskarte Sauna;50;0;0;0;0;0;0;0;10;7;0 (companion_price ist hier 0 und nicht „null“, da der Eintritt 0€ beträgt)
Eingabetext:
„Spaßbad 2 Stunden: Kinder: 3€, Schwerbehinderte: 4€, Schwerbehinderte mit mindestens 70 GdB oder Merkzeichen „H“ 2€, Normaler Eintritt: 6€, Begleitpersonen für Eingeschränkte: 50% Rabatt“
Korrekte Ausgabe (eigentlich als JSON): 
Spaßbad 2h;50;0;0;0;0;0;0;0;6;3;3 
Spaßbad 2h;70;0;0;0;0;0;1;0;6;2;3 
(companion_price wurde berechnet: 50% vom Normalpreis sind 3€)
Falls es einen zusätzlichen Rabatt für Rollstuhlfahrer gibt: "Rollstuhlfahrer zahlen nur 50€ statt 100€ für eine Jahreskarte“ 
Dann wird es ein separates JSON-Objekt im Array geben: 
Jahreskarte für Rollstuhlfahrer;50;0;0;0;0;0;0;0;100;50;null (companion_price ist hier „null“, da es keine Informationen für Begleiter gibt, für Rollstuhlfahrer-Zeilen ist der GdB immer mindestens 50)
Wenn es kein expliziter Gdb genannt wird ist der GdB 50, wenn dort SCHWERbehindert steht. Wenn dort nur etwas wie "Behindert" oder "eingeschränkt" steht, ist der GdB 20.'''


email_extraction_prompt_with_comparison = '''Ich werde dir gleich einen Text mit E-Mail-Adressen und einen zughörigen Firmenname geben, aus welchem du EINE E-Mail-Adresse herausfiltern sollst, von der du denkst, die am ehesten zur geschäftlichen Kontaktaufnahme mit der Lokalität geeignet ist.
Geschäftliche Kontaktaufnahme heißt, eine potentielle Zusammenarbeit bzw. eine Anfrage, ob man Daten der Website nutzen und daher sind Adressen wie presse@... oder datenschutz@... nicht geeignet. Adressen wie info@... oder kontakt@... sind in der Regel gut geeignet.
Also auch keine Adressen, an die ich schreiben würde, wenn ich ein normaler Besucher wäre, der eine Frage hat.
Dazu erhältst du eine bestehende E-Mail-Adresse, die du mit der herausgefilterten E-Mail-Adresse vergleichen sollst. NUR wenn du deine extrahierte Mail-Adresse als besser geeignet erachtest, gibst du sie zurück.
Bitte überprüfe kritisch, ob die E-Mail-Adresse wirklich zur Kontaktaufnahme geeignet ist und zum entsprechenden Unternehmen gehört.
Es kann sein, dass die Emails durch das Extrahieren aus einer Website sehr lang, unübersichtlich und somit keine echten Adressen sind. Dann ist deine Aufgabe, eine valide, sinnvolle E-Mail-Adresse herauszufiltern.
Gib mir ausschließlich die E-Mail-Adresse zurück. Keine weiteren Informationen, Wörter oder E-Mail-Adressen. 
Denk dir auf keinen Fall eine E-Mail-Adresse aus.
Wenn du keine E-Mail-Adresse herausfiltern kannst oder die bestehende besser geeignet ist, gib mir bitte "keine E-Mail-Adresse" zurück.
Wenn die bestehende unter den anderen Adressen gegeben ist und sie die beste ist, gib mir diese Adresse und nimm nicht eine andere. 
Das gesamte Ziel ist, von allen verfügbaren Adressen die beste herauszufiltern.
Hier kommt der Text, die Mail-Adresse und der Firmenname:
'''

email_extraction_prompt_without_comparison = '''Ich werde dir gleich einen Text mit E-Mail-Adressen und einen zughörigen Firmenname geben, aus welchem du EINE E-Mail-Adresse herausfiltern sollst, von der du denkst, die am ehesten zur geschäftlichen Kontaktaufnahme mit der Lokalität geeignet ist.
Geschäftliche Kontaktaufnahme heißt, eine potentielle Zusammenarbeit und daher sind Adressen wie presse@... oder datenschutz@... nicht geeignet. Adressen wie info@... oder kontakt@... sind in der Regel gut geeignet.
Also auch keine Adressen, an die ich schreiben würde, wenn ich ein normaler Besucher wäre, der eine Frage hat.
Bitte überprüfe kritisch, ob die E-Mail-Adresse wirklich zur Kontaktaufnahme geeignet ist und zum entsprechenden Unternehmen gehört.
Es kann sein, dass die Emails durch das Extrahieren aus einer Website sehr lang, unübersichtlich und somit keine echten Adressen sind. Dann ist deine Aufgabe, eine valide, sinnvolle E-Mail-Adresse herauszufiltern.
Gib mir ausschließlich die E-Mail-Adresse zurück. Keine weiteren Informationen, Wörter oder E-Mail-Adressen. 
Denk dir auf keinen Fall eine E-Mail-Adresse aus.
Wenn du keine E-Mail-Adresse herausfiltern kannst oder die bestehende besser geeignet ist, gib mir bitte "keine E-Mail-Adresse" zurück.
Wenn die bestehende unter den anderen Adressen gegeben ist und sie die beste ist, gib mir diese Adresse und nimm nicht eine andere. 
Das gesamte Ziel ist, von allen verfügbaren Adressen die beste herauszufiltern.
Hier kommt der Text, die Mail-Adresse und der Firmenname:
'''



def execute_with_retries(query_func, description, ids=None, max_retries=3):
    """Executes a Supabase query with retry logic and logs the result."""
    for attempt in range(max_retries):
        try:
            response = query_func()
            print(f"{description} successful (attempt {attempt + 1})")
            return response
        except Exception as e:
            print(f"Error in {description}, attempt {attempt + 1}: {e}")
            time.sleep(2)
    
    print(f"Failed to execute {description} after {max_retries} attempts")
    if ids:
        print(f"Manual check required for IDs: {ids}")
    raise Exception(f"Error in {description}")


In [ ]:
all_locality_discounts_df = pd.DataFrame()
all_localities_with_discounts_df = pd.DataFrame()
all_localities_without_discounts_df = pd.DataFrame()
all_localities_without_email_df = pd.DataFrame()
all_localities_with_errors_df = pd.DataFrame()

batch_size = 10
for i in range(0, len(fetched_localities_df), batch_size):

    print(f"New batch starting at index {i} with size th size of {batch_size} localities.")
    # DataFrame für valide Unternehmen mit Rabatten anlegen
    localities_with_discounts_df = pd.DataFrame(columns=['locality_id', 'website_email'])

    # DataFrame für ermittelte Rabatte anlegen
    locality_discounts_df = pd.DataFrame(columns=['locality_id', 'name_of_option', 'degree_of_disability', 'mark_ag', 'mark_b', 'mark_bl', 'mark_g', 'mark_gl', 'mark_h', 'mark_tb', 'standard_price', 'discounted_price', 'companion_price'])

    # DataFrame für Unternehmen ohne Rabatten anlegen
    localities_without_discounts_df = pd.DataFrame(columns=['locality_id'])

    # DataFrame für unvollständige und nicht brauchbare Unternehmen anlegen
    localities_with_errors_df = pd.DataFrame(columns=['locality_id', 'error'])
    
    batch = fetched_localities_df.iloc[i:i + batch_size]  

    for index, row in batch.iterrows():

        print("\n" + "\n" + "_" * 50 + "\n" + "_" * 50 + "\n")
        print(f"Processing Locality: {row['name']}")

        # Inhalt von Website abfragen
        try:
            crawl_result = fetch_content_from_website(row)
        except Exception as e:
            continue

        if crawl_result['is_valid'] == False:
            print(f"\nFollowing error occurred: " + crawl_result['error'])

            # Falls die Abfrage aus mehreren Gründen invalid ist, dem DataFrame für invalide Lokalitäten hinzufügen
            localities_with_errors_df.loc[len(localities_with_errors_df)] = [crawl_result['id'], crawl_result['error']]
            continue

        # Inhalt von Website bereinigen
        cleaned_pricing_content = clean_content(crawl_result['pricing_content'], False)

        if len(cleaned_pricing_content) > 20000:
            localities_with_errors_df.loc[len(localities_with_errors_df)] = [crawl_result['id'], 'too long']
            print(f"\nFollowing error occurred: too long ({len(cleaned_pricing_content)} characters)")
            continue

        print(f"\n\nCleaned pricing content:\n{cleaned_pricing_content}")

        
        cleaned_contact_email_adresses = clean_content(crawl_result['contact_email_adresses'], False)
        print(f"Cleaned Mails: {cleaned_contact_email_adresses}. Raw: {crawl_result['contact_email_adresses']}")

        if len(cleaned_pricing_content) < 10:
            print(f"\nFollowing error occurred: l;")
            # Falls der bereinigte Inhalt weniger als 10 Zeichen hat, ins DataFrame für invalide Lokalitäten 
            localities_with_errors_df.loc[len(localities_with_errors_df)] = [crawl_result['id'], 'l;']
            continue

        initial_extraction_response = call_api(True, discount_extraction_from_website_prompt_GEMINI, cleaned_pricing_content, 'gemini')

        initial_extraction_response_text = initial_extraction_response.summarized_discounts
    
        if initial_extraction_response is None:
            print(f'\nWhen calling the Gemini API, an error occurred. Skipping this request. The Locality will remain untouched in the database for a next try.')
            continue  


        if initial_extraction_response.has_discounts == False:
            print(f"\nNo discounts for: {row['name']} (going to next locality)")
            localities_without_discounts_df.loc[len(localities_without_discounts_df)] = [row['locality_id']]
            continue

        print(f"\n\nInitial summarization of discounts by Gemini: ")
        print(initial_extraction_response_text)    

        if row['email_from_osm'] is None and cleaned_contact_email_adresses is None:
            email_from_website = None

        if cleaned_contact_email_adresses is not None:
            if row['email_from_osm'] is not None:
                second_prompt = f"{email_extraction_prompt_with_comparison} \n Bestehende Mail-Adresse: {row['email_from_osm']} \n Firmenname: {row['name']}"
            elif row['email_from_osm'] is None:
                second_prompt = f"{email_extraction_prompt_without_comparison} \n Firmenname: {row['name']}"

            email_extraction_response_text = call_api(False, second_prompt,cleaned_contact_email_adresses, 'gemini')

            if email_extraction_response_text is None:
                print(f'\nWhen calling the Gemini API, an error occurred. Skipping this request. The Locality will remain untouched in the database for a next try.')
                continue

            if '@' in email_extraction_response_text and row['email_from_osm'] is None:
            
                email_from_website = email_extraction_response_text.replace(" ", "").replace("\n", "")

                if row['email_from_osm'] == email_extraction_response_text and row['email_from_osm'] is not None:
                    print(f"\nEmail address from OSM ({email_from_website}) is the best identified email address.")
                else:
                    print(f"\nBest email address identified from website by Gemini: {email_from_website}" )
            else: 
                email_from_website = None
        else: 
            email_from_website = None
            print(f"No email address found on website for: " + row['name'])
    
        validation_response_json = call_api(True,validate_gemini_response_prompt_CHATGPT, initial_extraction_response_text, 'openai', json_format=validation_json_format)

        if validation_response_json is None:
            print(f'\nWhen calling the OpenAI API, an error occurred. Skipping this request. The Locality will remain untouched in the database for a next try.')
            continue

        if validation_response_json['has_discounts_for_disabled'] == False:
            print(f"\nNo discounts for: {row['name']} (going to next locality)")
            localities_without_discounts_df.loc[len(localities_without_discounts_df)] = [row['locality_id']]
            continue

        final_extraction_response_json = call_api(True, discount_extraction_from_api_response_prompt_CHATGPT, initial_extraction_response_text, 'openai',                json_format=extraction_json_format)

        if final_extraction_response_json is None:
            print(f'\nWhen calling the OpenAI API, an error occurred. Skipping this request. The Locality will remain untouched in the database for a next try.')
            continue

        print(f"\n Structurized discounts by ChatGPT: ")
        for discount in final_extraction_response_json['discount_options']:
            print(discount)

        for discount in final_extraction_response_json['discount_options']:
            if not discount['standard_price'] < 1:
                locality_discounts_df.loc[len(locality_discounts_df)] = [row['locality_id'], discount['name_of_option'], discount['degree_of_disability'], discount['mark_ag'], discount['mark_b'], discount['mark_bl'], discount['mark_g'], discount['mark_gl'], discount['mark_h'], discount['mark_tb'], discount['standard_price'], discount['discounted_price'], discount['companion_price']]

        if len(locality_discounts_df[locality_discounts_df['locality_id'] == row['locality_id']]) != 0:
            localities_with_discounts_df.loc[len(localities_with_discounts_df)] = [row['locality_id'], email_from_website]


    # 1. Update localities with errors
    if not localities_with_errors_df.empty:
        # Create a dictionary for updates (assuming all rows need the same update)
        for _, row in localities_with_errors_df.iterrows():
            execute_with_retries(lambda: supabase.table('Locality')
                        .update({"error_code": row["error"]})
                        .eq("locality_id", row["locality_id"])
                        .execute(),
                        "Updating locality with error", row["locality_id"])

        all_localities_with_errors_df = pd.concat([all_localities_with_errors_df, localities_with_errors_df])

    # 2. Update localities without discounts
    if not localities_without_discounts_df.empty:
        ids_no_discounts = localities_without_discounts_df["locality_id"].tolist()
        
        # Execute the update to set 'has_discounts' to False for all relevant localities
        execute_with_retries(lambda: supabase.table('Locality')
                            .update({"has_discounts": False})
                            .in_("locality_id", ids_no_discounts)
                            .execute(), 
                            "Updating localities without discounts", ids_no_discounts)
        
        all_localities_without_discounts_df = pd.concat([all_localities_without_discounts_df, localities_without_discounts_df])



    # 3. Update localities with discounts
    if not localities_with_discounts_df.empty:
        # Create updates for all the rows, the values of the fields will be the same for all
        discount_updates = [{
            "has_discounts": True,
            "email_from_website": row['website_email']
        } for _, row in localities_with_discounts_df.iterrows()]

        ids = localities_with_discounts_df["locality_id"].tolist()
        # Execute the update to set 'has_discounts' to True and set other fields for the relevant localities
        execute_with_retries(lambda: supabase.table('Locality')
                            .update(discount_updates)
                            .in_("locality_id", ids)
                            .execute(), 
                            "Updating localities with discounts", ids)

                    
        all_localities_with_discounts_df = pd.concat([all_localities_with_discounts_df, localities_with_discounts_df])

    # 4. Insert locality discounts (batch insert)
    if not locality_discounts_df.empty:
        # Prepare the list of inserts for multiple rows
        discount_inserts = [{"locality_id": row["locality_id"],
                            "name_of_option": row["name_of_option"],
                            "degree_of_disability": row["degree_of_disability"],
                            "mark_ag": row["mark_ag"],
                            "mark_b": row["mark_b"],
                            "mark_bl": row["mark_bl"],
                            "mark_g": row["mark_g"],
                            "mark_gl": row["mark_gl"],
                            "mark_h": row["mark_h"],
                            "mark_tb": row["mark_tb"],
                            "standard_price": row["standard_price"],
                            "discounted_price": row["discounted_price"],
                            "companion_price": row["companion_price"],
                            "added_on": time.strftime('%Y-%m-%d'),
                            "confirmed_by_locality": False}
                            for _, row in locality_discounts_df.iterrows()]
        
        # Execute the insert for all the discount entries at once
        execute_with_retries(lambda: supabase.table('LocalityDiscount')
                            .insert(discount_inserts)
                            .execute(), 
                            "Inserting locality discounts", locality_discounts_df["locality_id"].tolist())

        all_locality_discounts_df = pd.concat([all_locality_discounts_df, locality_discounts_df])

    print("All updates and inserts completed. Moving on to the next batch...")
    print("_" * 50)

New batch starting at index 0 with size th size of 10 localities.


__________________________________________________
__________________________________________________

Processing Locality: Geschäftsstelle TSV Reinbek


__________________________________________________
__________________________________________________

Processing Locality: Kunstwerk Carlshütte


__________________________________________________
__________________________________________________

Processing Locality: Galileo Wissenswelt - Ausstellung Übersee + Dunkelexperiment


2025-05-26 21:12:10,400:INFO - AFC is enabled with max remote calls: 10.


Fehlende Links auf Mainpage der Website: https://galileo-fehmarn.de/ 


Cleaned pricing content:
Galileo Wissenswelt - Insel Fehmarn | Museum und Sehenswürdigkeit top of pageDas grösste interaktive Museum im Norden.In der Galileo Wissenswelt erleben sie zwei Museen auf 6.000m² in zwei spannenden Themenwelten.Galileo FehmarnTechnikWelt - HyperbelTechnikWelt - BalanceGalileo Fehmarn1/19BioWelt: Vom Dino zum Menschen TechnikWelt: Physik zum Mitmachen Jeder Bereich kann einzeln oder zusammen mit dem Kombi-Ticket besucht werden. Das Kombi-Ticket gilt im Sommer für 4 und im Winter für 2 Museen. Es ist bis zu 3 Jahre gültig. So können sie alle anderen Museen auch an einem anderen Tag erkunden.ÖffnungszeitenJan.,Feb.,Mrz............................... ab 1.April, Mai, Juni.................... Juli + August................................ September................................... Oktober........................................ Nov. + Dez................................... Weihnachten + Silve

2025-05-26 21:12:11,338:INFO - AFC remote call 1 is done.



No discounts for: Galileo Wissenswelt - Ausstellung Übersee + Dunkelexperiment (going to next locality)


__________________________________________________
__________________________________________________

Processing Locality: Funtasia Adventure Minigolf
Beim Website-Aufruf gab einen Error: https://funtasia-golf.de/

Following error occurred: c;W


__________________________________________________
__________________________________________________

Processing Locality: Theaterschiff Batavia


...

Fehlende Links auf Mainpage der Website: https://freibad-weiche.de/ 


Cleaned pricing content:
Freibad - Freibad Zum Inhalt wechseln Ferienschwimmen im Sommer Endlich Ferienzeit! Das beliebte Ferienschwimmen mit dem Freibad Weiche bietet Kindern und Jugendlichen die perfekte Gelegenheit, schwimmen zu lernen, ihre Fähigkeiten zu verbessern und dabei jede Menge Spaß zu haben Anmeldung Ferienschwimmen Werden Sie jetzt Mitglied und unterstützen Sie uns. Vereinsanmeldung Anmeldung Ferienschwimmen ABTAUCHEN BEI 26°Ob an heißen Sommertagen oder auch an etwas kühleren Tagen, das beheizte Freibad-Weiche lädt bei 26° zum Abtauchen ein. In den Sommermonaten ist das Bad täglich für Jung und Alt geöffnet. Warteliste Schlüsselbaden FERIENSCHWIMMENGemeinsam mit der Stadt Flensburg bieten wir ein Ferienschwimmen für Schüler:innen der Grundschulen der Stadt Flensburg an. Für Kinder mit Wohnsitz in Flensburg ist die Teilnahme kostenfrei. FREIBAD WEICHE Das Freibad Weiche war ursprünglich ein Feuerlösch

2025-05-26 21:17:36,558:INFO - AFC remote call 1 is done.
2025-05-26 21:17:36,559:INFO - AFC is enabled with max remote calls: 10.




Initial summarization of discounts by Gemini: 
Menschen mit einer Beeinträchtigung mit dem Merkzeichen B im Schwerbehindertenausweis können eine Begleitperson kostenfrei mitnehmen.

Eintrittspreise:
* Tageskarte:
  * Kinder (bis 14 Jahre): 2.00 €
  * Erwachsene (ab 15 Jahre): 3.00 €
* 11er Karte:
  * Kinder (bis 14 Jahre): 20.00 €
  * Erwachsene (ab 15 Jahre): 30.00 €
* Saisonkarte (bis 14 Jahre): 40.00 €


2025-05-26 21:17:37,249:INFO - AFC remote call 1 is done.



Best email address identified from website by Gemini: info@jugendtreff-weiche.de


In [ ]:
# text = input()
# if not text == 'sus123':
#     raise ValueError
# supabase.table('Locality').update({"error_code": None, "has_discounts":None, "allowed_after_request":None, "email_from_website":None}).eq("email_sent", False).execute()


2025-05-26 18:49:00,848:INFO - HTTP Request: PATCH https://pzxkdkuejjfgxgtkbubm.supabase.co/rest/v1/Locality?email_sent=eq.False "HTTP/1.1 200 OK"


APIResponse[TypeVar](data=[{'locality_id': '192207964', 'name': 'Well-You', 'website': 'https://wellyou.eu/', 'street': 'Feldstraße', 'house_number': '7a', 'city': 'Kiel', 'post_code': 24105, 'latitude': '54.3298064', 'longitude': '10.1387498', 'phone_number': None, 'wheelchair_accessible': False, 'opening_hour': None, 'error_code': None, 'email_from_osm': None, 'locality_type': 'Sportzentrum', 'has_discounts': None, 'email_sent': False, 'allowed_after_request': None, 'email_from_website': None, 'osm_type': 'sports_centre', 'last_allowed_at': None, 'email_greeting': None, 'nominalized_name': None, 'hash_code': '2dde08cd8bdc03c72dd8', 'last_disallowed_at': None, 'description': None}, {'locality_id': '192208197', 'name': 'Lady Fitness', 'website': 'https://ladyfitness.de/', 'street': 'Mittelstraße', 'house_number': '1', 'city': 'Kiel', 'post_code': 24103, 'latitude': '54.3292359', 'longitude': '10.1317737', 'phone_number': None, 'wheelchair_accessible': False, 'opening_hour': None, 'erro